# 卷积流程讲解
## stride和padding
<img src="./notebook_image_dir/Snipaste_2025-10-24_20-39-38.png">

In [ ]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

## 画幅大小
### 带padding
<img src="./notebook_image_dir/Snipaste_2025-10-26_17-40-44.png">

$w_{out} = n_w - k_w + 1 + p_w$

$h_{out} = n_h - k_h + 1 + p_h$

### 带stride
<img src="./notebook_image_dir/Snipaste_2025-10-26_17-42-34.png">

$ out_w =  [(n_w-k_w+p_w)/s_w+1] $

$ out_h =  [(n_h-k_h+p_h)/s_h+1] $

# 激活函数的添加位置

# 模型参数量计算
<img src="./notebook_image_dir/Snipaste_2025-10-24_20-56-11.png">

In [ ]:
# 参数量问题
# 卷积核大小为k*l，输入通道数为i，输出通道数为o，则参数量为(k*l)*i*o

# 钩子函数register_full_backward_hook实现Grad-CAM（绘图、包含图像反标准化）

In [ ]:
# 这里顺带也把反标准化做一下
# 见前一节

# 二分类损失函数

In [ ]:
# 需要用到BCELoss函数，pytorch提供了内置概率映射的转换BCEWithLogitsLoss
# 用法有点区别
# 模型更改
"""
res_net = nn.Sequential(b1, nn.Sequential(*b2), nn.Sequential(*b3), nn.Sequential(*b4), nn.Sequential(*b5),
                        # 自适应卷积，前两维1*1
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 1))   # (batch, 1)
"""

"""
loss = nn.BCEWithLogitsLoss()
for epoch in range(epochs):
    res_net.train()
    total_loss, total_acc, count = 0, 0, 0
    # 训练总损失、训练精度、训练经过的样本数
    start_time = time.time()
    for X, y in train_loader:
        X = X.to('mps')
        y.dtype = torch.float
        y = y.to('mps')
        y = y.reshape(-1, 1)
        y_hat = res_net(X)

        l = loss(y_hat, y)
        total_loss += l
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

"""

# nn.Sequential管道和手动控制计算的区别

In [ ]:
import torch.nn as nn

import torch
# 线性张量流
model = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 30),
    nn.ReLU()
)

# 涉及分支运算的时候，自定义类并且在forward函数中实现。或者语意上强调某几个小块块是构成一个逻辑上最小粒度控制器的情况下，用自定义类
# 诸如中途切换张量形状，用到.squeeze()/.view()等情况，可能还需要重复使用某一层

class NoneLinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3),
            nn.ReLU()
        )
        self.classifier = nn.Linear(128, 10)
        
    def forward(self, x):
        features = self.backbone(x)
        
        # 条件判断
        if self.training:
            # 训练时的特殊处理
            features = features + torch.randn_like(features) * 0.1
            
        # 故意一个复杂分支，seq能做但是要在构建seq的时候做判断
        branch1 = features.mean(dim=[2, 3])
        branch2 = features.max(dim=2)[0].max(dim=2)[0]
        
        combined = branch1 + branch2
        output = self.classifier(combined)
        
        return output, features  # 多输出

# nn.Sequential容器和nn.ModuleList、nn.ModuleDict的区别

# 模型参数的保存与读取

In [ ]:
# 见上一节末尾

# 参数字典
<img src="./notebook_image_dir/parameter_dict_running_result.png">